In [ ]:
#importing libraries
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pandas import DataFrame

In [ ]:
traingen = ImageDataGenerator(rescale = 1./255, horizontal_flip=True, vertical_flip=True, validation_split=0.2)
testgen = ImageDataGenerator()
train_gen = traingen.flow_from_directory(directory = "../input/plant-seedlings-classification/train",target_size=(256,256), color_mode = 'rgb',subset = 'training', class_mode = 'categorical', shuffle = True, batch_size = 32,seed = 2 )
valid_gen = traingen.flow_from_directory(directory = "../input/plant-seedlings-classification/train",target_size=(256,256), color_mode = 'rgb',subset = 'validation', class_mode = 'categorical', shuffle = True,batch_size = 32, seed = 2 )


In [ ]:
test_gen = testgen.flow_from_directory(
    directory="../input/testplants/test_reordered",
    target_size=(256,256),
    color_mode="rgb",
    batch_size=1,
    class_mode=None,
    shuffle=False,
    seed=2
)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
model = keras.Sequential([
    
    layers.Conv2D(filters=32, kernel_size=3, activation="relu", padding='same', input_shape=[256,256, 3]),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.BatchNormalization(),
    
    layers.Conv2D(filters=64, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Conv2D(filters=128, kernel_size=3, activation="relu", padding='same'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.BatchNormalization(),

    layers.Flatten(),
    layers.Dense(units=512, activation="relu"),
    layers.Dropout(0.2),
    layers.Dense(units=128, activation="relu"),
    layers.Dense(12, 'softmax')
                  ])
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(epsilon=0.01),
    loss='categorical_crossentropy',
    metrics=['categorical_accuracy']
)
model.summary()

In [ ]:
#checkpoint = tf.keras.callbacks.ModelCheckpoint('model.h5',save_best_only=True,monitor='val_accuracy',mode='max', verbose = 1)

In [ ]:
history = model.fit(
    train_gen,
    steps_per_epoch=3803//32,
    epochs=20,
    validation_data = valid_gen,
    validation_steps = 947//32,
    #callbacks = [checkpoint],
    verbose = 1,
)

In [ ]:
model.save("plant_class5.h5")

In [ ]:
#recon=tf.keras.models.load_model('./plant_class2.h5')

In [ ]:
STEP_SIZE_TEST=test_gen.n//test_gen.batch_size
test_gen.reset()
pred=model.predict(test_gen,
steps=STEP_SIZE_TEST,
verbose=1)

In [ ]:
predicted_class_indices=np.argmax(pred,axis=1)

In [ ]:
labels = (train_gen.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]

In [ ]:
predictions

In [ ]:
filenames=test_gen.filenames
results=pd.DataFrame({"file":filenames,
                      "species":predictions})
results.to_csv("results5.csv",index=False)